This loader fetches the text transcript of Youtube videos using the youtube_transcript_api Python package.

https://llamahub.ai/l/youtube_transcript


In [1]:
import os

In [2]:
os.chdir('/home/sankar')

In [3]:
mpath = "/home/sankar/Downloads/mistral-7b-openorca.Q5_K_M.gguf"

In [4]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
from llama_index.readers import YoutubeTranscriptReader

In [5]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=mpath,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/sankar/Downloads/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama

!pip install llama-index chromadb --quiet
!pip install chromadb
!pip install sentence-transformers
!pip install pydantic==1.10.11

https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html

https://docs.llamaindex.ai/en/latest/examples/vector_stores/ChromaIndexDemo.html



!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [6]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [ ]:
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [7]:
# create client and a new collection
#chroma_client = chromadb.EphemeralClient()
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("quickstart")

# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")



/home/sankar/anaconda3/envs/llamacpp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 777/777 [00:00<00:00, 1.56MB/s]
model.safetensors: 100%|██████████| 438M/438M [02:05<00:00, 3.50MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 811kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 4.39MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 13.1MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 231kB/s]


# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [15]:
loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=xS4dkhBGh1Y&list=PL6ZR-sD5vIlw3o7IBFfLsvC6Hw72h-LLK&index=2'])

documents = YoutubeTranscriptReader("https://www.youtube.com/watch?v=xS4dkhBGh1Y&list=PL6ZR-sD5vIlw3o7IBFfLsvC6Hw72h-LLK&index=2").load_data()

In [53]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model)
index1 = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)


In [54]:
# Query Data
query_engine1 = index1.as_query_engine()
response = query_engine1.query("please summarize in 200 words or less?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      40.16 ms /   110 runs   (    0.37 ms per token,  2738.84 tokens per second)
llama_print_timings: prompt eval time =   15399.18 ms /  2300 tokens (    6.70 ms per token,   149.36 tokens per second)
llama_print_timings:        eval time =   19044.51 ms /   109 runs   (  174.72 ms per token,     5.72 tokens per second)
llama_print_timings:       total time =   34712.42 ms


<b>
The East India Company's initial focus on spices led them into competition with the Dutch, who had superior ships and financial resources. By 1630, the English lost this competition, leading to a shift in focus from spices to textiles. The English began trading cotton from Gujarat, which was considered a luxury item at the time. This trade eventually led to the development of the textile industry in England, and the eventual decline of the East India Company's monopoly on spice trade.</b>

In [55]:
# Query Data
query_engine1 = index1.as_query_engine()
response = query_engine1.query("please summarize in 500 words or less?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      67.35 ms /   180 runs   (    0.37 ms per token,  2672.69 tokens per second)
llama_print_timings: prompt eval time =    1213.62 ms /    12 tokens (  101.13 ms per token,     9.89 tokens per second)
llama_print_timings:        eval time =   31390.12 ms /   179 runs   (  175.36 ms per token,     5.70 tokens per second)
llama_print_timings:       total time =   33020.94 ms


<b>
The story begins with Sir James Lancaster who is not a very successful sea captain. He loses his flotilla and many of his crew to cannibals on a previous voyage. However, he manages to bump into a Portuguese ship which he takes everything from and goes home. This marks the beginning of a tricky 30 years for the British as they compete against the Dutch in the spice trade. The Dutch, being a new nation with better ships and sea captains, eventually win and the English are left with a muddy island called Manhattan.

Realizing that the days of the spice trade have passed, the English shift their focus to textiles. They start importing cotton from Bengal, which was considered a luxury item in ancient times. The British begin wearing cotton instead of wool, which is unsuitable for the tropics.</b>

In [56]:
# Query Data
query_engine1 = index1.as_query_engine()
response = query_engine1.query("Who are the two people speaking in this episode?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      21.78 ms /    61 runs   (    0.36 ms per token,  2800.99 tokens per second)
llama_print_timings: prompt eval time =   14979.63 ms /  2278 tokens (    6.58 ms per token,   152.07 tokens per second)
llama_print_timings:        eval time =   10311.30 ms /    60 runs   (  171.86 ms per token,     5.82 tokens per second)
llama_print_timings:       total time =   25431.53 ms


<b>1) The person who is speaking in this episode is not explicitly named but they discuss the history of the East India Company and its founder, customer Smythe. 2) Another speaker mentioned in the context is historian William, who provides information about the company's early days and its founder.</b>

In [57]:
response = query_engine1.query("Ine one sentence what is this epidose about?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      12.67 ms /    34 runs   (    0.37 ms per token,  2683.29 tokens per second)
llama_print_timings: prompt eval time =   15192.13 ms /  2285 tokens (    6.65 ms per token,   150.41 tokens per second)
llama_print_timings:        eval time =    5616.89 ms /    33 runs   (  170.21 ms per token,     5.88 tokens per second)
llama_print_timings:       total time =   20890.29 ms


<b> This episode discusses the history of Indian artifacts and treasures that were looted during British colonization and are now housed in a private collection in Wales.</b>

In [58]:
response = query_engine1.query("The blurb for this podcasts says the following.In the opening episode, William and Anita discuss the rise of The East India Company, exploring how a small corporation founded in Tudor England - with only a handful of employees - came to rule India. With that in mind, give a 5 sentence paragraph for each 5 minute of this 60 minute podcast")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      60.72 ms /   164 runs   (    0.37 ms per token,  2701.10 tokens per second)
llama_print_timings: prompt eval time =   15495.84 ms /  2344 tokens (    6.61 ms per token,   151.27 tokens per second)
llama_print_timings:        eval time =   28330.51 ms /   163 runs   (  173.81 ms per token,     5.75 tokens per second)
llama_print_timings:       total time =   44208.21 ms


<b>
In this podcast, William and Anita delve into the fascinating history of The East India Company, which began as a small corporation in Tudor England. They discuss how this company, founded by auditor-turned-entrepreneur Sir George Auditor Smith, managed to grow rapidly and eventually conquer vast territories in India. Despite being outnumbered and having fewer resources compared to their opponents, the English adventurers were able to subdue North India through a series of strategic battles. The podcast also touches upon the role of privateers, who were licensed by the state to loot treasure ships, in helping the East India Company establish its presence in the region. Overall, this engaging discussion provides listeners with an insightful look into the rise and influence of The East India Company in India.</b>

In [59]:
from llama_index.node_parser.text import SentenceSplitter

In [60]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [61]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [62]:
from llama_index.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [63]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [64]:
vector_store.add(nodes)

['7b5044b1-2cd6-44b1-aa18-3d01ea24eada',
 'c6ce0ee4-af53-4f17-95d4-546d2e010f7f',
 '712e2d31-9bf1-4b4e-8279-9c54544f8f8a',
 'f9e2ee83-eec3-4373-a91b-827d092d6216',
 '5a42490d-3559-4725-ae37-9a78a3d0a970',
 'bc386350-9c75-4750-916c-9bf07c18e808',
 '2cf7bb84-0bab-4d5f-b8af-29f43bce9859',
 '16b54ef7-4875-4430-93d5-50111066349c',
 '134bc013-f0a7-4476-84ab-3d55fe1922d5',
 'f0fd968d-4140-4d1d-9da0-ea5d3ce8f08c',
 'd3c182a0-0744-429a-8478-8a0cff94e85c',
 'de0159bd-a164-4428-9359-beb7b922cbb5',
 '47953010-b812-41bd-9321-0c42cd2dedb9',
 '5e0346e5-b733-42fe-9281-c6b4f8e7aa8c',
 'dc7aa922-07b6-45c5-bcda-b4d315010e49',
 '5147d09c-7a11-4fdc-a7f2-71c18fa6d9fb',
 '3832dc89-110c-4f79-a02a-89c4e77fc2ec',
 'f7934960-c8c0-4d8d-82e2-0745d0f7b162']

In [65]:
query_str = "What did Clive acheive?"


In [66]:
query_embedding = embed_model.get_query_embedding(query_str)

In [67]:
# construct vector store query
from llama_index.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [68]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


and
everyone laughs and he's off there and
there's oh oh Clive but that explains
where we started Paris Castle stuff not
quite not right no no no nearly so what
happens then is that Clive gets his
million and eight years later all the
people that he has put into power as
puppies again rise up against English
because the English have behaved so
incredibly badly and basically you know
killed the goose that was laying the
golden egg in just eight years they lay
waste to Bengal by asset stripping it
and there is another big uh Act of
resistance and not just the nawab of
Bengal this time who's now called me a
customer man actually put in by by Clive
but uh also the nawab of of avid which
is basically uttar Pradesh who's called
shuja odala and the Mughal Emperor
himself
all meet at the Battle of buxar and they
take on uh the East India Company but
again the company has used the money
that it's gathered to enormously
increase the size of its sepoyamin it's
bought an enormous number of sepoys 

In [69]:
from llama_index.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [70]:
from llama_index import QueryBundle
from llama_index.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a chroma vector store."""

    def __init__(
        self,
        vector_store: ChromaVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [71]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [72]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever, service_context=service_context
)

In [73]:
query_str = "What did Clive acheive?"

In [74]:
response.response

'\nIn this podcast, William and Anita delve into the fascinating history of The East India Company, which began as a small corporation in Tudor England. They discuss how this company, founded by auditor-turned-entrepreneur Sir George Auditor Smith, managed to grow rapidly and eventually conquer vast territories in India. Despite being outnumbered and having fewer resources compared to their opponents, the English adventurers were able to subdue North India through a series of strategic battles. The podcast also touches upon the role of privateers, who were licensed by the state to loot treasure ships, in helping the East India Company establish its presence in the region. Overall, this engaging discussion provides listeners with an insightful look into the rise and influence of The East India Company in India.'

In [75]:
query_str = "What did Clive acheive?"

response = query_engine.query(query_str)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      19.98 ms /    56 runs   (    0.36 ms per token,  2803.08 tokens per second)
llama_print_timings: prompt eval time =    6288.52 ms /  1084 tokens (    5.80 ms per token,   172.38 tokens per second)
llama_print_timings:        eval time =    8861.53 ms /    55 runs   (  161.12 ms per token,     6.21 tokens per second)
llama_print_timings:       total time =   15274.07 ms


In [76]:
display(Markdown(f"<b>{response}</b>"))

<b>
Clive achieved significant victories in India, which led to him being awarded a million pounds. He established the East India Company as a powerful force in India. His actions set the stage for the company's transformation from a trading organization to an imperial territorial power.</b>

In [77]:
query_str = "what years is this episode covering"

response = query_engine.query(query_str)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =       2.18 ms /     6 runs   (    0.36 ms per token,  2751.03 tokens per second)
llama_print_timings: prompt eval time =   15197.84 ms /  2274 tokens (    6.68 ms per token,   149.63 tokens per second)
llama_print_timings:        eval time =     838.47 ms /     5 runs   (  167.69 ms per token,     5.96 tokens per second)
llama_print_timings:       total time =   16055.93 ms


In [78]:
display(Markdown(f"<b>{response}</b>"))

<b>1590s</b>

In [79]:
query_str = "Write a bruief biography of Clive of India strictly based on the facts and thoughts experessed in this podcast?"

response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      50.08 ms /   140 runs   (    0.36 ms per token,  2795.53 tokens per second)
llama_print_timings: prompt eval time =    2083.06 ms /   422 tokens (    4.94 ms per token,   202.59 tokens per second)
llama_print_timings:        eval time =   22125.79 ms /   139 runs   (  159.18 ms per token,     6.28 tokens per second)
llama_print_timings:       total time =   24516.83 ms


<b>
Clive of India, also known as Robert Clive, was a prominent British military officer who played a significant role in shaping the British Empire's presence in India. Born in England, he went on to make a name for himself through his military accomplishments and strategic prowess. As a product of the British Empire, Clive's life and career were deeply intertwined with the expansionist policies of his time. His actions in India contributed to the establishment of British rule over large parts of the Indian subcontinent. Despite the complexities of his legacy, Clive of India remains an important figure in understanding the history of the British Empire and its impact on India.</b>

In [80]:
query_str = "Summarize in 300 words, the last 15 minutes of the podcast. ?"

response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      92.31 ms /   252 runs   (    0.37 ms per token,  2730.05 tokens per second)
llama_print_timings: prompt eval time =   15096.97 ms /  2289 tokens (    6.60 ms per token,   151.62 tokens per second)
llama_print_timings:        eval time =   43108.52 ms /   251 runs   (  171.75 ms per token,     5.82 tokens per second)
llama_print_timings:       total time =   58791.97 ms


<b>
The podcast discusses the East India Company's journey to control a vast territory like India. It begins with the founder of the company, customer or auditor Smythe, who was an accountant and ran the customs of London. He had the idea that if the Dutch could get to the East Indies (Indonesia) and buy spices by cutting out all middlemen, so can the English. He appealed to the patriotism of rich investors and called in privateers or robbers who were licensed by the state to loot treasure ships.

The podcast then moves on to discuss how England cut itself off from the continent after the Reformation and rivalry with Spain and Portugal. The Dutch, who were Protestants like the English, had made a fortune after their pioneering visit to the spice lands. This led to the idea that the East India Company could also subdue all opposition in North India.

The podcast then circles back to the question of how a group of adventurers managed to control such a vast country as India despite being outnumbered and having less strength in arms. The answer lies in their determination, patriotism, and the support from their home country, England.</b>

In [81]:
response = query_engine.query("what do we know about Anita Anand?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      25.13 ms /    69 runs   (    0.36 ms per token,  2746.16 tokens per second)
llama_print_timings: prompt eval time =    2075.39 ms /   408 tokens (    5.09 ms per token,   196.59 tokens per second)
llama_print_timings:        eval time =   10742.83 ms /    68 runs   (  157.98 ms per token,     6.33 tokens per second)
llama_print_timings:       total time =   12976.45 ms


<b>
From the given context, we know that Anita Anand is a person of Indian origin who was born in Essex. She identifies herself as an "Indian in Britain". She is one half of the podcast duo Empire along with William Durham pool. The podcast discusses the British Empire and its impact on India.</b>

In [82]:
query_str = "What do we know about William Dalrymple"

response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      37.05 ms /   102 runs   (    0.36 ms per token,  2753.26 tokens per second)
llama_print_timings: prompt eval time =   15028.94 ms /  2276 tokens (    6.60 ms per token,   151.44 tokens per second)
llama_print_timings:        eval time =   17342.21 ms /   101 runs   (  171.71 ms per token,     5.82 tokens per second)
llama_print_timings:       total time =   32605.21 ms


<b>
William Dalrymple is a British historian, author, and journalist who specializes in Indian history and culture. He was born in Scotland but has spent much of his life living and working in India. His writing focuses on the historical interactions between Britain and India, particularly during the colonial period. Some of his notable works include "The Age of Kali: The Eighteenth Century in India" and "White Mughals: Love & Betrayal in 18th Century India."</b>

In [83]:
query_str = "What do we glean about William Dalrymple from this podcast"

response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      39.49 ms /   107 runs   (    0.37 ms per token,  2709.62 tokens per second)
llama_print_timings: prompt eval time =   15328.45 ms /  2282 tokens (    6.72 ms per token,   148.87 tokens per second)
llama_print_timings:        eval time =   18758.21 ms /   106 runs   (  176.96 ms per token,     5.65 tokens per second)
llama_print_timings:       total time =   34360.42 ms


<b>
From the given context, it is clear that William Dalrymple is a historian who has a deep interest in the history of the East India Company and its founder, customer Smythe. He appears to be knowledgeable about the historical events surrounding the company's formation and its early ventures into the spice trade. His enthusiasm for the subject matter is evident as he discusses the fascinating story of how an English company managed to subdue a vast territory like India despite being outnumbered and having fewer resources.</b>

In [84]:
retriever.retrieve(query_str)

[NodeWithScore(node=TextNode(id_='4b64d2fc-0f91-475c-8855-e9de5b9f1457', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a', text="is\nthe moment before that episode this is\nthe moment when an English company first\ngets its hands on this incredibly Rich\nterritory and it's done it in two stages\nfirst to be and we'll we'll deal with\nthis over the course of the next uh uh\nhour of this podcast but the uh the\nfirst stage is the battle plasti the\nsecond stage of the battle of buxar and\nthis signing of this document takes\nplace immediately after the Second\nBattle okay when the East India company\nhas now basically subdued all opposition\nand North India we okay I think that's\ncan we Circle back to that because I\nstill want to know and this is a\nquestion that Indians ask all the time\nhow is it that in numbers we outnumbered\nin strength of arms we w

In [85]:
response = query_engine1.query("Who are the two people speaking in this episode?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      13.63 ms /    37 runs   (    0.37 ms per token,  2714.20 tokens per second)
llama_print_timings: prompt eval time =    1703.83 ms /    15 tokens (  113.59 ms per token,     8.80 tokens per second)
llama_print_timings:        eval time =    6331.54 ms /    36 runs   (  175.88 ms per token,     5.69 tokens per second)
llama_print_timings:       total time =    8132.60 ms


<b>1) The person who is speaking most of the time seems to be an historian named William. 2) The other speaker is presumably the host or interviewer of the podcast.</b>

In [86]:
response = query_engine1.query("Summarise the last 15 minutes of the poidcast?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      46.63 ms /   128 runs   (    0.36 ms per token,  2745.19 tokens per second)
llama_print_timings: prompt eval time =    1884.43 ms /    20 tokens (   94.22 ms per token,    10.61 tokens per second)
llama_print_timings:        eval time =   21755.63 ms /   127 runs   (  171.30 ms per token,     5.84 tokens per second)
llama_print_timings:       total time =   23939.37 ms


<b>
The podcast discusses the history of the East India Company, which was founded by Sir Auditor Smith in the late 1590s. The company initially aimed to establish a presence in Indonesia and beyond, rather than India. Sir Auditor Smith appealed to the patriotism of wealthy investors, many of whom were privateers or pirates, to fund his venture. This was during a time when England had cut itself off from the continent due to the Reformation and tensions with Catholic Spain and Portugal. The Dutch, who were also Protestant, had recently made a fortune by visiting the spice lands.</b>

In [87]:
response = query_engine.query("Who are the two people speaking in this episode?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      40.63 ms /   110 runs   (    0.37 ms per token,  2707.29 tokens per second)
llama_print_timings: prompt eval time =    1368.74 ms /    15 tokens (   91.25 ms per token,    10.96 tokens per second)
llama_print_timings:        eval time =   18928.65 ms /   109 runs   (  173.66 ms per token,     5.76 tokens per second)
llama_print_timings:       total time =   20554.78 ms


<b>1) The person who is speaking in this podcast is not explicitly named but he seems to be a historian or an expert on the topic of the East India Company. He provides detailed historical information about the company's founding, its initial focus on Indonesia and the Spice Islands, and its eventual expansion into India. 2) The other person who is speaking in this podcast is presumably the host or interviewer, who engages with the historian and asks questions to elicit more information and context about the East India Company's history.</b>

In [88]:
response = query_engine1.query("The podcast is an hour long.In th einitial 5 minutes or so the hosts introduce themselves. Can you name thema dn give detauils about them from the first 10 minutes only")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      12.86 ms /    36 runs   (    0.36 ms per token,  2799.16 tokens per second)
llama_print_timings: prompt eval time =     909.09 ms /    47 tokens (   19.34 ms per token,    51.70 tokens per second)
llama_print_timings:        eval time =    6180.90 ms /    35 runs   (  176.60 ms per token,     5.66 tokens per second)
llama_print_timings:       total time =    7182.30 ms


<b>
The podcast does not provide any information about the hosts in the initial 5 minutes or so. The context provided does not mention any details about the hosts of the podcast.</b>

In [89]:
response = query_engine1.query("can you reproduce the transcripts of the first 5 minutes of the podcast")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      88.58 ms /   256 runs   (    0.35 ms per token,  2889.98 tokens per second)
llama_print_timings: prompt eval time =    1981.49 ms /    21 tokens (   94.36 ms per token,    10.60 tokens per second)
llama_print_timings:        eval time =   44776.48 ms /   255 runs   (  175.59 ms per token,     5.69 tokens per second)
llama_print_timings:       total time =   47380.91 ms


<b>
The transcript for the first five minutes of the podcast is as follows:

[Introduction]

Speaker 1: Welcome to the podcast where we explore the fascinating history of India and its interactions with the world. Today, we're going to talk about the East India Company and how it came to control a vast territory in India. So let's start at the very beginning.

[Start of the podcast]

Speaker 1: The moment before that episode this is the moment when an English company first gets its hands on this incredibly Rich territory and it's done it in two stages first to be and we'll we'll deal with this over the course of the next uh uh hour of this podcast but the uh the first stage is the battle plasti the second stage of the battle of buxar and this signing of this document takes place immediately after the Second Battle okay when the East India company has now basically subdued all opposition and North India we okay I think that's can we Circle back to that because I still want to know and this is a question that Indians ask all the time how is it that in numbers we outnumbered in strength of arms we were more powerful</b>

In [90]:
response = query_engine.query("can you reproduce the transcripts of the first 5 minutes of the podcast")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      87.53 ms /   256 runs   (    0.34 ms per token,  2924.58 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   44314.49 ms /   256 runs   (  173.10 ms per token,     5.78 tokens per second)
llama_print_timings:       total time =   44931.41 ms


<b>
The transcripts for the first five minutes of the podcast are as follows:

"Context information is below.
---------------------
is
the moment before that episode this is
the moment when an English company first
gets its hands on this incredibly Rich
territory and it's done it in two stages
first to be and we'll we'll deal with
this over the course of the next uh uh
hour of this podcast but the uh the
first stage is the battle plasti the
second stage of the battle of buxar and
this signing of this document takes
place immediately after the Second
Battle okay when the East India company
has now basically subdued all opposition
and North India we okay I think that's
can we Circle back to that because I
still want to know and this is a
question that Indians ask all the time
how is it that in numbers we outnumbered
in strength of arms we were more
powerful together and yet
a group of adventurers managed to subdue
and control a country as vast as India
so can we go right right right right
back to the first birth whales the
Crad</b>

In [91]:
response = query_engine1.query("acn you reproduce th etranscript for the first 30 seconds of the podcast?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      87.84 ms /   256 runs   (    0.34 ms per token,  2914.56 tokens per second)
llama_print_timings: prompt eval time =    2315.26 ms /    25 tokens (   92.61 ms per token,    10.80 tokens per second)
llama_print_timings:        eval time =   43853.69 ms /   255 runs   (  171.98 ms per token,     5.81 tokens per second)
llama_print_timings:       total time =   46767.48 ms


<b>
Yes, here is the transcript for the first 30 seconds of the podcast:

"Is the moment before that episode this is the moment when an English company first gets its hands on this incredibly Rich territory and it's done it in two stages first to be and we'll we'll deal with this over the course of the next uh uh hour of this podcast but the uh the first stage is the battle plasti the second stage of the battle of buxar and this signing of this document takes place immediately after the Second Battle okay when the East India company has now basically subdued all opposition and North India we okay I think that's can we Circle back to that because I still want to know and this is a question that Indians ask all the time how is it that in numbers we outnumbered in strength of arms we were more powerful together and yet a group of adventurers managed to subdue and control a country as vast as India so can we go right right right right back to the first birth whales the Cradle whales of what would become the East India Company and this really fascinating man who I just love historian William called if we are going to be called what we do customer Smy</b>

In [92]:
response = query_engine.query("acn you reproduce th etranscript for the first 30 seconds of the podcast?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      88.36 ms /   256 runs   (    0.35 ms per token,  2897.30 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   44290.74 ms /   256 runs   (  173.01 ms per token,     5.78 tokens per second)
llama_print_timings:       total time =   44901.73 ms


<b>
The transcript for the first 30 seconds of the podcast is as follows:
"Is the moment before that episode this is the moment when an English company first gets its hands on this incredibly Rich territory and it's done it in two stages first to be and we'll we'll deal with this over the course of the next uh uh hour of this podcast but the uh the first stage is the battle plasti the second stage of the battle of buxar and this signing of this document takes place immediately after the Second Battle okay when the East India company has now basically subdued all opposition and North India we okay I think that's can we Circle back to that because I still want to know and this is a question that Indians ask all the time how is it that in numbers we outnumbered in strength of arms we were more powerful together and yet a group of adventurers managed to subdue and control a country as vast as India so can we go right right right right back to the first birth whales the Cradle whales of what would become the East India Company and this really fascinating man who I just love historian William called if we are going to be called what we do customer Smythe can</b>

In [97]:
response = query_engine.query("The hosts of thepodcast mention their name when they sign off ath the end. Can you tell me their names.one of them says That's all from me Anita Anand. and any other details about Anita?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


<b>
The hosts of the podcast are Anita Anand and William Durham pool. Anita Anand is an Indian in Britain, born in Essex, and has made most of her life in India.</b>


llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      14.92 ms /    43 runs   (    0.35 ms per token,  2881.84 tokens per second)
llama_print_timings: prompt eval time =    1160.51 ms /    14 tokens (   82.89 ms per token,    12.06 tokens per second)
llama_print_timings:        eval time =    6659.52 ms /    42 runs   (  158.56 ms per token,     6.31 tokens per second)
llama_print_timings:       total time =    7919.42 ms


In [93]:
documents[0]


Document(id_='a299f0a7-f2aa-4453-ab27-8c0aea0f4e27', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='6342d048795a6ef115d90eb0f0b8b6940172d15d4503dd6b1c33bdd055b58d49', text="[Music]\nhello and welcome to a brand new podcast\ncalled Empire with me Anita Anand and me\nWilliam Durham pool now um in a funny\nway we are both products of Empire I\nmean you could say very simplistically\nI'm the Indian in Britain I'm an of\nIndian origin born in Essex but you know\nnevertheless you know what I mean and\nI'm I'm the Brit that's lived most made\nout of life in India oh and why are we\ndoing this why are we actually doing\nthis well I think it's a very\ninteresting moment to think about Empire\num it's particularly an interesting\nmoment to think about the British Empire\nand the British Empire in India because\nthis is something which I suddenly feel\ngot sort of tucked in an attic in 1947.\nthe British moved on they joined th

In [98]:
response = query_engine.query("Why does Anita want to make this podcast?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      28.11 ms /    80 runs   (    0.35 ms per token,  2846.37 tokens per second)
llama_print_timings: prompt eval time =   15813.43 ms /  2282 tokens (    6.93 ms per token,   144.31 tokens per second)
llama_print_timings:        eval time =   13622.07 ms /    79 runs   (  172.43 ms per token,     5.80 tokens per second)
llama_print_timings:       total time =   29626.51 ms


<b>
Anita wants to make this podcast because she feels it's an interesting moment to think about Empire, particularly the British Empire in India. She believes that the topic of Empire has been tucked away and forgotten in recent history, and she wants to bring attention to it through the podcast. Additionally, Anita notes that Empire is a controversial word, which adds another reason for making the podcast.</b>

In [99]:
response = query_engine.query("What does clive say in the parliament?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2547.08 ms
llama_print_timings:      sample time =      17.04 ms /    47 runs   (    0.36 ms per token,  2758.70 tokens per second)
llama_print_timings: prompt eval time =   15423.08 ms /  2313 tokens (    6.67 ms per token,   149.97 tokens per second)
llama_print_timings:        eval time =    8074.19 ms /    46 runs   (  175.53 ms per token,     5.70 tokens per second)
llama_print_timings:       total time =   23629.18 ms


<b>
In Parliament, Clive says, "My lords, there was a prostrate City at my feet. The bankers waited on my pleasure. My lords, I was astonished at my own moderation."</b>

'''
(llamacpp) sankar@sankar-ml:~$ conda list
# packages in environment at /home/sankar/anaconda3/envs/llamacpp:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
accelerate                0.25.0                   pypi_0    pypi
aiofiles                  23.2.1                   pypi_0    pypi
aiohttp                   3.9.1                    pypi_0    pypi
aiosignal                 1.3.1                    pypi_0    pypi
aiostream                 0.5.2                    pypi_0    pypi
altair                    5.2.0                    pypi_0    pypi
annotated-types           0.6.0                    pypi_0    pypi
anyio                     3.7.1                    pypi_0    pypi
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py311h5eee18b_0  
asgiref                   3.7.2                    pypi_0    pypi
asttokens                 2.0.5              pyhd3eb1b0_0  
async-lru                 2.0.4           py311h06a4308_0  
attrs                     23.1.0          py311h06a4308_0  
babel                     2.11.0          py311h06a4308_0  
backcall                  0.2.0              pyhd3eb1b0_0  
backoff                   2.2.1                    pypi_0    pypi
bcrypt                    4.1.1                    pypi_0    pypi
beautifulsoup4            4.12.2          py311h06a4308_0  
bitsandbytes              0.41.3.post2             pypi_0    pypi
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0  
bottleneck                1.3.5           py311hbed6279_0  
brotlipy                  0.7.0           py311h5eee18b_1002  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2023.08.22           h06a4308_0  
cachetools                5.3.2                    pypi_0    pypi
certifi                   2023.11.17      py311h06a4308_0  
cffi                      1.15.1          py311h5eee18b_3  
charset-normalizer        2.0.4              pyhd3eb1b0_0  
chroma-hnswlib            0.7.3                    pypi_0    pypi
chromadb                  0.4.18                   pypi_0    pypi
click                     8.1.7                    pypi_0    pypi
colorama                  0.4.6                    pypi_0    pypi
coloredlogs               15.0.1                   pypi_0    pypi
comm                      0.1.2           py311h06a4308_0  
contourpy                 1.2.0                    pypi_0    pypi
cryptography              41.0.3          py311hdda0065_0  
cuda-cudart               11.8.89                       0    nvidia
cuda-cupti                11.8.87                       0    nvidia
cuda-libraries            11.8.0                        0    nvidia
cuda-nvrtc                11.8.89                       0    nvidia
cuda-nvtx                 11.8.86                       0    nvidia
cuda-runtime              11.8.0                        0    nvidia
cudatoolkit               11.8.0               h6a678d5_0    anaconda
cycler                    0.12.1                   pypi_0    pypi
dataclasses-json          0.6.3                    pypi_0    pypi
debugpy                   1.6.7           py311h6a678d5_0  
decorator                 5.1.1              pyhd3eb1b0_0  
defusedxml                0.7.1              pyhd3eb1b0_0  
deprecated                1.2.14                   pypi_0    pypi
diskcache                 5.6.3                    pypi_0    pypi
distro                    1.8.0                    pypi_0    pypi
executing                 0.8.3              pyhd3eb1b0_0  
fastapi                   0.104.1                  pypi_0    pypi
ffmpeg                    4.3                  hf484d3e_0    pytorch
ffmpy                     0.3.1                    pypi_0    pypi
filelock                  3.9.0           py311h06a4308_0  
flatbuffers               23.5.26                  pypi_0    pypi
fonttools                 4.47.0                   pypi_0    pypi
freetype                  2.12.1               h4a9f257_0  
frozenlist                1.4.0                    pypi_0    pypi
fsspec                    2023.12.1                pypi_0    pypi
giflib                    5.2.1                h5eee18b_3  
gmp                       6.2.1                h295c915_3  
gmpy2                     2.1.2           py311hc9b5ff0_0  
gnutls                    3.6.15               he1e5248_0  
google-auth               2.25.2                   pypi_0    pypi
googleapis-common-protos  1.62.0                   pypi_0    pypi
gradio                    4.12.0                   pypi_0    pypi
gradio-client             0.8.0                    pypi_0    pypi
greenlet                  3.0.2                    pypi_0    pypi
grpcio                    1.60.0                   pypi_0    pypi
h11                       0.14.0                   pypi_0    pypi
httpcore                  1.0.2                    pypi_0    pypi
httptools                 0.6.1                    pypi_0    pypi
httpx                     0.25.2                   pypi_0    pypi
huggingface-hub           0.19.4                   pypi_0    pypi
humanfriendly             10.0                     pypi_0    pypi
idna                      3.4             py311h06a4308_0  
importlib-metadata        6.11.0                   pypi_0    pypi
importlib-resources       6.1.1                    pypi_0    pypi
intel-openmp              2023.1.0         hdb19cb5_46305  
ipykernel                 6.25.0          py311h92b7b1e_0  
ipython                   8.15.0          py311h06a4308_0  
jedi                      0.18.1          py311h06a4308_1  
jinja2                    3.1.2           py311h06a4308_0  
joblib                    1.3.2                    pypi_0    pypi
jpeg                      9e                   h5eee18b_1  
json5                     0.9.6              pyhd3eb1b0_0  
jsonpatch                 1.33                     pypi_0    pypi
jsonpointer               2.4                      pypi_0    pypi
jsonschema                4.19.2          py311h06a4308_0  
jsonschema-specifications 2023.7.1        py311h06a4308_0  
jupyter-lsp               2.2.0           py311h06a4308_0  
jupyter_client            8.6.0           py311h06a4308_0  
jupyter_core              5.5.0           py311h06a4308_0  
jupyter_events            0.8.0           py311h06a4308_0  
jupyter_server            2.10.0          py311h06a4308_0  
jupyter_server_terminals  0.4.4           py311h06a4308_1  
jupyterlab                4.0.8           py311h06a4308_0  
jupyterlab_pygments       0.1.2                      py_0  
jupyterlab_server         2.25.1          py311h06a4308_0  
kiwisolver                1.4.5                    pypi_0    pypi
kubernetes                28.1.0                   pypi_0    pypi
lame                      3.100                h7b6447c_0  
langchain                 0.0.344                  pypi_0    pypi
langchain-core            0.0.13                   pypi_0    pypi
langsmith                 0.0.75                   pypi_0    pypi
lcms2                     2.12                 h3be6417_0  
ld_impl_linux-64          2.38                 h1181459_1  
lerc                      3.0                  h295c915_0  
libcublas                 11.11.3.6                     0    nvidia
libcufft                  10.9.0.58                     0    nvidia
libcufile                 1.7.2.10                      0    nvidia
libcurand                 10.3.3.141                    0    nvidia
libcusolver               11.4.1.48                     0    nvidia
libcusparse               11.7.5.86                     0    nvidia
libdeflate                1.17                 h5eee18b_0  
libffi                    3.4.4                h6a678d5_0  
libgcc-ng                 11.2.0               h1234567_1  
libgomp                   11.2.0               h1234567_1  
libiconv                  1.16                 h7f8727e_2  
libidn2                   2.3.4                h5eee18b_0  
libnpp                    11.8.0.86                     0    nvidia
libnvjpeg                 11.9.0.86                     0    nvidia
libpng                    1.6.39               h5eee18b_0  
libsodium                 1.0.18               h7b6447c_0  
libstdcxx-ng              11.2.0               h1234567_1  
libtasn1                  4.19.0               h5eee18b_0  
libtiff                   4.5.1                h6a678d5_0  
libunistring              0.9.10               h27cfd23_0  
libuuid                   1.41.5               h5eee18b_0  
libwebp                   1.2.4                h11a3e52_1  
libwebp-base              1.2.4                h5eee18b_1  
llama-cpp-python          0.2.20                   pypi_0    pypi
llama-index               0.9.13                   pypi_0    pypi
lz4-c                     1.9.4                h6a678d5_0  
markdown-it-py            3.0.0                    pypi_0    pypi
markupsafe                2.1.1           py311h5eee18b_0  
marshmallow               3.20.1                   pypi_0    pypi
matplotlib                3.8.2                    pypi_0    pypi
matplotlib-inline         0.1.6           py311h06a4308_0  
mdurl                     0.1.2                    pypi_0    pypi
mistune                   2.0.4           py311h06a4308_0  
mkl                       2023.1.0         h213fc3f_46343  
mkl-service               2.4.0           py311h5eee18b_1  
mkl_fft                   1.3.6           py311ha02d727_1  
mkl_random                1.2.2           py311ha02d727_1  
mmh3                      4.0.1                    pypi_0    pypi
monotonic                 1.6                      pypi_0    pypi
mpc                       1.1.0                h10f8cd9_1  
mpfr                      4.0.2                hb69a4c5_1  
mpmath                    1.3.0           py311h06a4308_0  
multidict                 6.0.4                    pypi_0    pypi
mypy-extensions           1.0.0                    pypi_0    pypi
nbclient                  0.8.0           py311h06a4308_0  
nbconvert                 7.10.0          py311h06a4308_0  
nbformat                  5.9.2           py311h06a4308_0  
ncurses                   6.4                  h6a678d5_0  
nest-asyncio              1.5.8                    pypi_0    pypi
nettle                    3.7.3                hbbd107a_1  
networkx                  3.1             py311h06a4308_0  
nltk                      3.8.1                    pypi_0    pypi
notebook                  7.0.6           py311h06a4308_0  
notebook-shim             0.2.3           py311h06a4308_0  
numexpr                   2.8.4           py311h65dcdc2_1  
numpy                     1.25.2          py311h08b1b3b_0  
numpy-base                1.25.2          py311hf175353_0  
oauthlib                  3.2.2                    pypi_0    pypi
onnxruntime               1.16.3                   pypi_0    pypi
openai                    1.3.8                    pypi_0    pypi
openh264                  2.1.1                h4ff587b_0  
openssl                   3.0.12               h7f8727e_0  
opentelemetry-api         1.21.0                   pypi_0    pypi
opentelemetry-exporter-otlp-proto-common 1.21.0                   pypi_0    pypi
opentelemetry-exporter-otlp-proto-grpc 1.21.0                   pypi_0    pypi
opentelemetry-instrumentation 0.42b0                   pypi_0    pypi
opentelemetry-instrumentation-asgi 0.42b0                   pypi_0    pypi
opentelemetry-instrumentation-fastapi 0.42b0                   pypi_0    pypi
opentelemetry-proto       1.21.0                   pypi_0    pypi
opentelemetry-sdk         1.21.0                   pypi_0    pypi
opentelemetry-semantic-conventions 0.42b0                   pypi_0    pypi
opentelemetry-util-http   0.42b0                   pypi_0    pypi
orjson                    3.9.10                   pypi_0    pypi
overrides                 7.4.0           py311h06a4308_0  
packaging                 23.2                     pypi_0    pypi
pandas                    2.0.3           py311ha02d727_0  
pandocfilters             1.5.0              pyhd3eb1b0_0  
parso                     0.8.3              pyhd3eb1b0_0  
pexpect                   4.8.0              pyhd3eb1b0_3  
pickleshare               0.7.5           pyhd3eb1b0_1003  
pillow                    9.4.0           py311h6a678d5_0  
pip                       23.2.1          py311h06a4308_0  
platformdirs              3.10.0          py311h06a4308_0  
posthog                   3.1.0                    pypi_0    pypi
prometheus_client         0.14.1          py311h06a4308_0  
prompt-toolkit            3.0.36          py311h06a4308_0  
protobuf                  4.25.1                   pypi_0    pypi
psutil                    5.9.0           py311h5eee18b_0  
ptyprocess                0.7.0              pyhd3eb1b0_2  
pulsar-client             3.3.0                    pypi_0    pypi
pure_eval                 0.2.2              pyhd3eb1b0_0  
pyasn1                    0.5.1                    pypi_0    pypi
pyasn1-modules            0.3.0                    pypi_0    pypi
pycparser                 2.21               pyhd3eb1b0_0  
pydantic                  2.5.3                    pypi_0    pypi
pydantic-core             2.14.6                   pypi_0    pypi
pydub                     0.25.1                   pypi_0    pypi
pygments                  2.15.1          py311h06a4308_1  
pyopenssl                 23.2.0          py311h06a4308_0  
pyparsing                 3.1.1                    pypi_0    pypi
pypika                    0.48.9                   pypi_0    pypi
pysocks                   1.7.1           py311h06a4308_0  
python                    3.11.5               h955ad1f_0  
python-dateutil           2.8.2              pyhd3eb1b0_0  
python-dotenv             1.0.0                    pypi_0    pypi
python-fastjsonschema     2.16.2          py311h06a4308_0  
python-json-logger        2.0.7           py311h06a4308_0  
python-multipart          0.0.6                    pypi_0    pypi
python-tzdata             2023.3             pyhd3eb1b0_0  
pytorch                   2.0.1           py3.11_cuda11.8_cudnn8.7.0_0    pytorch
pytorch-cuda              11.8                 h7e8668a_5    pytorch
pytorch-mutex             1.0                        cuda    pytorch
pytz                      2022.7          py311h06a4308_0  
pyyaml                    6.0.1           py311h5eee18b_0  
pyzmq                     25.1.0          py311h6a678d5_0  
readline                  8.2                  h5eee18b_0  
referencing               0.30.2          py311h06a4308_0  
regex                     2023.10.3                pypi_0    pypi
requests                  2.31.0          py311h06a4308_0  
requests-oauthlib         1.3.1                    pypi_0    pypi
rfc3339-validator         0.1.4           py311h06a4308_0  
rfc3986-validator         0.1.1           py311h06a4308_0  
rich                      13.7.0                   pypi_0    pypi
rpds-py                   0.10.6          py311hb02cf49_0  
rsa                       4.9                      pypi_0    pypi
safetensors               0.4.1                    pypi_0    pypi
scikit-learn              1.3.2                    pypi_0    pypi
scipy                     1.11.4                   pypi_0    pypi
semantic-version          2.10.0                   pypi_0    pypi
send2trash                1.8.2           py311h06a4308_0  
sentence-transformers     2.2.2                    pypi_0    pypi
sentencepiece             0.1.99                   pypi_0    pypi
setuptools                68.0.0          py311h06a4308_0  
shellingham               1.5.4                    pypi_0    pypi
six                       1.16.0             pyhd3eb1b0_1  
sniffio                   1.2.0           py311h06a4308_1  
soupsieve                 2.5             py311h06a4308_0  
sqlalchemy                2.0.23                   pypi_0    pypi
sqlite                    3.41.2               h5eee18b_0  
stack_data                0.2.0              pyhd3eb1b0_0  
starlette                 0.27.0                   pypi_0    pypi
sympy                     1.11.1          py311h06a4308_0  
tbb                       2021.8.0             hdb19cb5_0  
tenacity                  8.2.3                    pypi_0    pypi
terminado                 0.17.1          py311h06a4308_0  
threadpoolctl             3.2.0                    pypi_0    pypi
tiktoken                  0.5.2                    pypi_0    pypi
tinycss2                  1.2.1           py311h06a4308_0  
tk                        8.6.12               h1ccaba5_0  
tokenizers                0.15.0                   pypi_0    pypi
tomlkit                   0.12.0                   pypi_0    pypi
toolz                     0.12.0                   pypi_0    pypi
torchaudio                2.0.2               py311_cu118    pytorch
torchtriton               2.0.0                     py311    pytorch
torchvision               0.15.2              py311_cu118    pytorch
tornado                   6.3.3           py311h5eee18b_0  
tqdm                      4.66.1                   pypi_0    pypi
traitlets                 5.7.1           py311h06a4308_0  
transformers              4.35.2                   pypi_0    pypi
typer                     0.9.0                    pypi_0    pypi
typing-extensions         4.9.0                    pypi_0    pypi
typing-inspect            0.9.0                    pypi_0    pypi
tzdata                    2023c                h04d1e81_0  
urllib3                   1.26.16         py311h06a4308_0  
uvicorn                   0.24.0.post1             pypi_0    pypi
uvloop                    0.19.0                   pypi_0    pypi
watchfiles                0.21.0                   pypi_0    pypi
wcwidth                   0.2.5              pyhd3eb1b0_0  
webencodings              0.5.1           py311h06a4308_1  
websocket-client          0.58.0          py311h06a4308_4  
websockets                11.0.3                   pypi_0    pypi
wheel                     0.38.4          py311h06a4308_0  
wrapt                     1.16.0                   pypi_0    pypi
xz                        5.4.2                h5eee18b_0  
yaml                      0.2.5                h7b6447c_0  
yarl                      1.9.4                    pypi_0    pypi
youtube-transcript-api    0.6.1                    pypi_0    pypi
zeromq                    4.3.4                h2531618_0  
zipp                      3.17.0                   pypi_0    pypi
zlib                      1.2.13               h5eee18b_0  
zstd                      1.5.5                hc292b87_0  
(llamacpp) sankar@sankar-ml:~$ pip list
Package                                  Version
---------------------------------------- ------------
accelerate                               0.25.0
aiofiles                                 23.2.1
aiohttp                                  3.9.1
aiosignal                                1.3.1
aiostream                                0.5.2
altair                                   5.2.0
annotated-types                          0.6.0
anyio                                    3.7.1
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.7.2
asttokens                                2.0.5
async-lru                                2.0.4
attrs                                    23.1.0
Babel                                    2.11.0
backcall                                 0.2.0
backoff                                  2.2.1
bcrypt                                   4.1.1
beautifulsoup4                           4.12.2
bitsandbytes                             0.41.3.post2
bleach                                   4.1.0
Bottleneck                               1.3.5
brotlipy                                 0.7.0
cachetools                               5.3.2
certifi                                  2023.11.17
cffi                                     1.15.1
charset-normalizer                       2.0.4
chroma-hnswlib                           0.7.3
chromadb                                 0.4.18
click                                    8.1.7
colorama                                 0.4.6
coloredlogs                              15.0.1
comm                                     0.1.2
contourpy                                1.2.0
cryptography                             41.0.3
cycler                                   0.12.1
dataclasses-json                         0.6.3
debugpy                                  1.6.7
decorator                                5.1.1
defusedxml                               0.7.1
Deprecated                               1.2.14
diskcache                                5.6.3
distro                                   1.8.0
executing                                0.8.3
fastapi                                  0.104.1
fastjsonschema                           2.16.2
ffmpy                                    0.3.1
filelock                                 3.9.0
flatbuffers                              23.5.26
fonttools                                4.47.0
frozenlist                               1.4.0
fsspec                                   2023.12.1
gmpy2                                    2.1.2
google-auth                              2.25.2
googleapis-common-protos                 1.62.0
gradio                                   4.12.0
gradio_client                            0.8.0
greenlet                                 3.0.2
grpcio                                   1.60.0
h11                                      0.14.0
httpcore                                 1.0.2
httptools                                0.6.1
httpx                                    0.25.2
huggingface-hub                          0.19.4
humanfriendly                            10.0
idna                                     3.4
importlib-metadata                       6.11.0
importlib-resources                      6.1.1
ipykernel                                6.25.0
ipython                                  8.15.0
jedi                                     0.18.1
Jinja2                                   3.1.2
joblib                                   1.3.2
json5                                    0.9.6
jsonpatch                                1.33
jsonpointer                              2.4
jsonschema                               4.19.2
jsonschema-specifications                2023.7.1
jupyter_client                           8.6.0
jupyter_core                             5.5.0
jupyter-events                           0.8.0
jupyter-lsp                              2.2.0
jupyter_server                           2.10.0
jupyter_server_terminals                 0.4.4
jupyterlab                               4.0.8
jupyterlab-pygments                      0.1.2
jupyterlab_server                        2.25.1
kiwisolver                               1.4.5
kubernetes                               28.1.0
langchain                                0.0.344
langchain-core                           0.0.13
langsmith                                0.0.75
llama_cpp_python                         0.2.20
llama-index                              0.9.13
markdown-it-py                           3.0.0
MarkupSafe                               2.1.1
marshmallow                              3.20.1
matplotlib                               3.8.2
matplotlib-inline                        0.1.6
mdurl                                    0.1.2
mistune                                  2.0.4
mkl-fft                                  1.3.6
mkl-random                               1.2.2
mkl-service                              2.4.0
mmh3                                     4.0.1
monotonic                                1.6
mpmath                                   1.3.0
multidict                                6.0.4
mypy-extensions                          1.0.0
nbclient                                 0.8.0
nbconvert                                7.10.0
nbformat                                 5.9.2
nest-asyncio                             1.5.8
networkx                                 3.1
nltk                                     3.8.1
notebook                                 7.0.6
notebook_shim                            0.2.3
numexpr                                  2.8.4
numpy                                    1.25.2
oauthlib                                 3.2.2
onnxruntime                              1.16.3
openai                                   1.3.8
opentelemetry-api                        1.21.0
opentelemetry-exporter-otlp-proto-common 1.21.0
opentelemetry-exporter-otlp-proto-grpc   1.21.0
opentelemetry-instrumentation            0.42b0
opentelemetry-instrumentation-asgi       0.42b0
opentelemetry-instrumentation-fastapi    0.42b0
opentelemetry-proto                      1.21.0
opentelemetry-sdk                        1.21.0
opentelemetry-semantic-conventions       0.42b0
opentelemetry-util-http                  0.42b0
orjson                                   3.9.10
overrides                                7.4.0
packaging                                23.2
pandas                                   2.0.3
pandocfilters                            1.5.0
parso                                    0.8.3
pexpect                                  4.8.0
pickleshare                              0.7.5
Pillow                                   9.4.0
pip                                      23.2.1
platformdirs                             3.10.0
posthog                                  3.1.0
prometheus-client                        0.14.1
prompt-toolkit                           3.0.36
protobuf                                 4.25.1
psutil                                   5.9.0
ptyprocess                               0.7.0
pulsar-client                            3.3.0
pure-eval                                0.2.2
pyasn1                                   0.5.1
pyasn1-modules                           0.3.0
pycparser                                2.21
pydantic                                 2.5.3
pydantic_core                            2.14.6
pydub                                    0.25.1
Pygments                                 2.15.1
pyOpenSSL                                23.2.0
pyparsing                                3.1.1
PyPika                                   0.48.9
PySocks                                  1.7.1
python-dateutil                          2.8.2
python-dotenv                            1.0.0
python-json-logger                       2.0.7
python-multipart                         0.0.6
pytz                                     2022.7
PyYAML                                   6.0.1
pyzmq                                    25.1.0
referencing                              0.30.2
regex                                    2023.10.3
requests                                 2.31.0
requests-oauthlib                        1.3.1
rfc3339-validator                        0.1.4
rfc3986-validator                        0.1.1
rich                                     13.7.0
rpds-py                                  0.10.6
rsa                                      4.9
safetensors                              0.4.1
scikit-learn                             1.3.2
scipy                                    1.11.4
semantic-version                         2.10.0
Send2Trash                               1.8.2
sentence-transformers                    2.2.2
sentencepiece                            0.1.99
setuptools                               68.0.0
shellingham                              1.5.4
six                                      1.16.0
sniffio                                  1.2.0
soupsieve                                2.5
SQLAlchemy                               2.0.23
stack-data                               0.2.0
starlette                                0.27.0
sympy                                    1.11.1
tenacity                                 8.2.3
terminado                                0.17.1
threadpoolctl                            3.2.0
tiktoken                                 0.5.2
tinycss2                                 1.2.1
tokenizers                               0.15.0
tomlkit                                  0.12.0
toolz                                    0.12.0
torch                                    2.0.1
torchaudio                               2.0.2
torchvision                              0.15.2
tornado                                  6.3.3
tqdm                                     4.66.1
traitlets                                5.7.1
transformers                             4.35.2
triton                                   2.0.0
typer                                    0.9.0
typing_extensions                        4.9.0
typing-inspect                           0.9.0
tzdata                                   2023.3
urllib3                                  1.26.16
uvicorn                                  0.24.0.post1
uvloop                                   0.19.0
watchfiles                               0.21.0
wcwidth                                  0.2.5
webencodings                             0.5.1
websocket-client                         0.58.0
websockets                               11.0.3
wheel                                    0.38.4
wrapt                                    1.16.0
yarl                                     1.9.4
youtube-transcript-api                   0.6.1
zipp                                     3.17.0
(llamacpp) sankar@sankar-ml:~$ 


'''
